In [2]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Associação entre empresas, tickers e palavras-chave para busca de notícias
companies = {
    'Petrobras': {'ticker': 'PETR4.SA', 'news_query': 'Petrobras'},
    'Weg': {'ticker': 'WEGE3.SA', 'news_query': 'Weg'},
    'C&A': {'ticker': 'CEAB3.SA', 'news_query': 'c%26a'}
}

# Coletando os dados dos últimos 12 meses para os tickers
dados = {company['ticker']: yf.download(company['ticker'], period="1y") for company in companies.values()}

# Função para buscar notícias no site brazil journal
def get_news(query):
    url = f"https://braziljournal.com/?s={query}"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        titles = soup.select(".boxarticle-infos-title")
        tags = soup.select(".boxarticle-infos-tag")
        links = soup.select(".boxarticle-infos-title a")

        news = []
        for title, tag, link in zip(titles[:3], tags[:3], links[:3]):
            news_title = title.get_text(strip=True)
            news_tag = tag.get_text(strip=True)
            news_link = link.get('href')
            news.append(html.A(f"{news_tag}: {news_title}", href=news_link, target='_blank'))

        return news
    else:
        return ["Falha ao obter a página"]

# Iniciando a aplicação do Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Label('Selecione a empresa:'),
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': company, 'value': company} for company in companies.keys()
        ],
        value='Petrobras'
    ),
    dcc.Graph(id='candlestick-chart'),
    html.Div(id='news-output')
])

# Callback para atualizar o gráfico com base na seleção do botão dropdown
@app.callback(
    Output('candlestick-chart', 'figure'),
    [Input('dropdown', 'value')]
)
def update_chart(selected_company):
    selected_company_info = companies[selected_company]
    selected_ticker = selected_company_info['ticker']

    fig = go.Figure(data=[go.Candlestick(
        x=dados[selected_ticker].index,
        open=dados[selected_ticker]['Open'],
        high=dados[selected_ticker]['High'],
        low=dados[selected_ticker]['Low'],
        close=dados[selected_ticker]['Close']
    )])

    fig.update_layout(
        title=f'Gráfico Candlestick - {selected_ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        showlegend=False,
        xaxis=dict(rangeslider=dict(visible=False))
    )

    return fig

# Callback para atualizar as notícias com base na seleção do mesmo botão dropdown
@app.callback(
    Output('news-output', 'children'),
    [Input('dropdown', 'value')]
)
def update_news(selected_company):
    selected_company_info = companies[selected_company]
    selected_news_query = selected_company_info['news_query']

    news = get_news(selected_news_query)
    news_html = [html.P(article, style={'margin-bottom': '10px'}) for article in news]
    return news_html

# Executando o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
